In [2]:
import cv2
import numpy as np
img = cv2.imread('Images/Sunshine_girl3.jpg',1)

In [4]:
# 结合蚁群算法的改进OSTU多阈值算法
np.seterr(invalid='ignore') #之所以加这一句是因为下面除法的时候会除零报错
import random
import numpy as np
def normalize_probabilities(probabilities):
    total = np.sum(probabilities)
    if total == 0:
        # 如果总和为零，返回均匀分布
        return np.ones_like(probabilities) / len(probabilities)
    normalized = probabilities / total
    # 将任何 NaN 值替换为均匀分布值
    normalized = np.where(np.isnan(normalized), 1.0 / len(probabilities), normalized)
    return normalized

def Fitness_Function(img,th1,th2):
    k1 = th1
    k2 = th2
    if(img.shape[2]==3)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    pixels = cv2.calcHist([img], [0], None, [256], [0, 256])                # 计算每个灰度级中所含像素数，返回的是一个（256,1）的数组
    p = pixels / (img.shape[0] * img.shape[1])                              # 获得每个灰度级中像素数占总像素数比例则我们获得了p_i的一个向量
    x=np.linspace(1,256,256)                                                # 灰度级像素范围定义，若定义为0～255，则求平均灰度级时会忽略第一个数据，所以我们定义为从1到256                                                 
    maxvar = 0                                                              # 最大类间类内方差
    mu_T = np.sum(x*p)                                                      # 平均灰度级
    step = 0
    a0 = np.sum(p[0:k1])
    a1 = np.sum(p[k1:k2])
    a2 = 1-a0-a1
    if(a0==0):
        r1 = 0
    else:
        mu_0 = np.dot(x[0:k1],p[0:k1])/a0
        r1 = a0*(mu_0-mu_T)**2
    if(a1==0):
        r2 = 0
    else:
        mu_1 = np.dot(x[k1:k2],p[k1:k2])/a1
        r2 = a1*(mu_1-mu_T)**2
    if(a2==0):
        r3 = 0
    else:
        mu_2 = np.dot(x[k2:256],p[k2:256])/a2
        r3 = a2*(mu_2-mu_T)**2
    sigma_b = r1+r2+r3
    m1 = np.dot(x[0:k1],p[0:k1])/a0
    m2 = np.dot(x[k1:k2],p[k1:k2])/a1
    m3 = np.dot(x[k2:256],p[k2:256])/a2
    sigma_in = np.sum(p[0:k1]*(x[0:k1]-m1)**2)+np.sum(p[k1:k2]*(x[k1:k2]-m2)**2) +np.sum(p[k2:256]*(x[k2:256]-m3)**2)
    vars = sigma_b/sigma_in
    return vars

# 伽马校正
def Gamma_correction(img,gamma):
    dst = np.array(255 * (img / 255) ** gamma, dtype='uint8')           # 先归一化处理， 再进行非线性变换，最后再映射到[0,255]
    # cv2.imshow('src',img)
    # cv2.imshow('dst',dst)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    return dst

# 通过直方图均衡化对Cr分量进行处理
def Hist_equalization_Cr(img):
    ycbcr = cv2.cvtColor(img, cv2.COLOR_BGR2YCR_CB)
    Cr = ycbcr[:,:,1]
    #Cr_equ = cv2.equalizeHist(Cr)
    clahe = cv2.createCLAHE(clipLimit=0.6, tileGridSize=(30,30))
    y_equ = ycbcr[:,:,0]
    Cr_equ = clahe.apply(Cr)
    cv2.imshow("1",Cr)
    cv2.imshow("2",Cr_equ)
    cv2.imwrite("Cr_equ.png",Cr_equ)
    ycbcr_equ = cv2.merge([y_equ,Cr_equ,ycbcr[:,:,2]])
    img_equ = cv2.cvtColor(ycbcr_equ, cv2.COLOR_YCR_CB2BGR)
    cv2.imshow("img",img)
    cv2.imshow("equ",img_equ)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return img_equ


In [5]:
img = Hist_equalization_Cr(img)
ycrcb = cv2.cvtColor(img,cv2.COLOR_BGR2YCrCb)      #   RGB->YCrCb  有没有转换公式呢？
(y,cr,cb) = cv2.split(ycrcb)                       # 图像分割，分别获取y,cr,br通道图像
cr1 = cv2.GaussianBlur(cr, (5, 5), 0) # # 高斯滤波, cr 是待滤波的源图像数据, (5,5)是值窗口大小, 0 是指根据窗口大小来计算高斯函数标准差
img = cr1
#def OTSU_MULTY_IMPROVE_ACO(img):
num_ants=10                 # 蚂蚁数量
max_iter=20               # 最大迭代次数
alpha=2                   # 信息素重要性因子
rho=0.2                     # 信息素挥发系数
Q= 4                       # 信息素强度常数  
pheromone = np.ones([256,2])    # 初始化信息素                                            # 初始化每个灰度值的信息素都为1
best_threshold = 0
best_maxvar = 0
ants = np.zeros((num_ants,2), dtype=int)
threshold_prob = np.ones([256,2])/256
iteration_result = np.zeros([255,2])

for iteration in range(max_iter):
    for i in range(num_ants):
        
        p1 = normalize_probabilities(threshold_prob[:, 0])                  #我们需要确保在归一化概率分布之前和之后都不会出现 NaN 值。此外，还需要处理概率分布为零的情况。
        ants[i][0] = np.random.choice(np.arange(0, 256), p=p1)                      # 第i只蚂蚁选择的阈值th1
        
        p2 = normalize_probabilities(threshold_prob[ants[i][0]:,1])
        ants[i][1] = np.random.choice(np.arange(ants[i][0], 256),p=p2)                      # 第i只蚂蚁选择的阈值th2
        th1 = ants[i][0]
        th2 = ants[i][1]
        maxvar = Fitness_Function(img,th1,th2)
        if maxvar > best_maxvar:
            best_maxvar = maxvar
            best_threshold = [th1,th2]
            
    pheromone = (1 - rho) * pheromone
    for t in range(1, 256):
        for i in range(0,2):
            pheromone_sum= np.sum(pheromone[:,i]**alpha)
            threshold_prob[t-1][i] = pheromone[t-1][i]**alpha/pheromone_sum      # 前往每一个灰度值的概率
    threshold_prob[:,0]/=np.sum(threshold_prob[:,0])                      # 重新归一化
    threshold_prob[:,1]/=np.sum(threshold_prob[:,1])                      # 重新归一化
    #print(threshold_prob)
    for i in range(num_ants):
        th = ants[i]
        maxvar = Fitness_Function(img,th[0],th[1])
        pheromone[th-1] += Q * maxvar/best_maxvar                          # 为每一只蚂蚁走过路径的信息素加权
        #print(pheromone[th-1])
    iteration_result[iteration][0] = iteration+1
    iteration_result[iteration][1] = best_maxvar
    print(f"第 {iteration+1} 代: 最佳阈值 {best_threshold}, 最大方差比 {best_maxvar}")
    #return best_threshold, best_maxvar, iteration_result




 

error: OpenCV(4.9.0) d:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.simd_helpers.hpp:92: error: (-2:Unspecified error) in function '__cdecl cv::impl::`anonymous-namespace'::CvtHelper<struct cv::impl::`anonymous namespace'::Set<3,4,-1>,struct cv::impl::A0x16764f4e::Set<1,-1,-1>,struct cv::impl::A0x16764f4e::Set<0,2,5>,4>::CvtHelper(const class cv::_InputArray &,const class cv::_OutputArray &,int)'
> Invalid number of channels in input image:
>     'VScn::contains(scn)'
> where
>     'scn' is 1
